In [167]:
cd "c:\Users\ecometto001\Documents\Personal\Tesis"

c:\Users\ecometto001\Documents\Personal\Tesis


In [168]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

plt.style.use('dark_background')
random_state=42

#### Leer los datos

In [169]:
tesis = pd.read_csv("Datos/Datos_Tesis_Septiembre2020.csv", na_values=' ')
tesis.rename({"TAS_noe":"TAS", "Morisky": "Adherencia"}, axis=1, inplace=True)
tesis.drop(["DSCVisita", "fechavisita", "fecha_visitabasal", "IMC_25", "TA_ALTA",
            "Cintura", "Cadera", "IMC_30", "ICC_alto", "Tabaquismo", "ant_HTA_trat",
            "n_drogas_actual_categ"],
           axis=1, inplace=True)
tesis["Sexo"] = tesis["Sexo"].map({1: 1, 2: 0})
tesis["Fuma"] = tesis["Fuma"].map({0: 0, 1: 1, 2: 1})

In [170]:
tesis.head()

,idPaciente,tpo_programa,TAS,Adherencia,Peso,Altura,IMC,DBT,Sexo,Edad,Fuma,ant_HTA,tas_basal,ICC
0,4017,0,106,1,72.0,154.0,30.0,1.0,0,51.0,0.0,1,139,0.943396
1,4017,1,116,1,72.0,154.0,30.0,1.0,0,51.0,0.0,1,139,0.943396
2,4017,2,123,1,72.0,154.0,30.0,1.0,0,51.0,0.0,1,139,0.943396
3,4017,3,130,1,72.0,154.0,30.0,1.0,0,51.0,0.0,1,139,0.943396
4,4017,4,121,1,72.0,154.0,30.0,1.0,0,51.0,0.0,1,139,0.943396


#### Valores faltantes

In [171]:
tesis.isnull().sum()

idPaciente        0
tpo_programa      0
TAS               0
Adherencia        0
Peso            252
Altura          256
IMC             260
DBT             215
Sexo              0
Edad            455
Fuma            215
ant_HTA           0
tas_basal         0
ICC             874
dtype: int64

#### Estadísticas básicas covariables continuas

In [172]:
tesis.describe()

,idPaciente,tpo_programa,TAS,Adherencia,Peso,Altura,IMC,DBT,Sexo,Edad,Fuma,ant_HTA,tas_basal,ICC
count,8592.000000,8592.000000,8592.000000,8592.000000,8340.000000,8336.000000,8332.000000,8377.000000,8592.000000,8137.000000,8377.000000,8592.000000,8592.000000,7718.000000
mean,16805.614758,5.738245,132.418645,0.842993,85.795084,163.854726,31.455833,0.147547,0.489525,59.048789,0.469500,0.900140,148.743017,0.944068
std,2012.288717,4.678866,14.207203,0.363828,17.413438,10.184401,5.494585,0.354671,0.499919,10.165842,0.499099,0.299831,19.307981,0.077841
min,4017.000000,0.000000,100.000000,0.000000,43.000000,0.000000,16.000000,0.000000,0.000000,20.000000,0.000000,0.000000,100.000000,0.588832
25%,16629.500000,2.000000,123.000000,1.000000,73.000000,157.000000,27.000000,0.000000,0.000000,53.000000,0.000000,1.000000,135.000000,0.894231
50%,17168.000000,5.000000,132.000000,1.000000,85.000000,164.000000,31.000000,0.000000,0.000000,59.000000,0.000000,1.000000,147.000000,0.944954
75%,17664.000000,9.000000,139.000000,1.000000,97.000000,171.000000,35.000000,0.000000,1.000000,66.000000,1.000000,1.000000,161.000000,1.000000
max,18666.000000,24.000000,198.000000,1.000000,180.000000,191.000000,54.000000,1.000000,1.000000,88.000000,1.000000,1.000000,198.000000,1.191667


Altura, tiene de valor mínimo 0, lo cual es imposible, por lo tanto voy a imputarlos con NaN

In [173]:
tesis.loc[tesis["Altura"] == 0, "Altura"] = np.nan

In [174]:
tesis.describe()

,idPaciente,tpo_programa,TAS,Adherencia,Peso,Altura,IMC,DBT,Sexo,Edad,Fuma,ant_HTA,tas_basal,ICC
count,8592.000000,8592.000000,8592.000000,8592.000000,8340.000000,8332.000000,8332.000000,8377.000000,8592.000000,8137.000000,8377.000000,8592.000000,8592.000000,7718.000000
mean,16805.614758,5.738245,132.418645,0.842993,85.795084,163.933389,31.455833,0.147547,0.489525,59.048789,0.469500,0.900140,148.743017,0.944068
std,2012.288717,4.678866,14.207203,0.363828,17.413438,9.532828,5.494585,0.354671,0.499919,10.165842,0.499099,0.299831,19.307981,0.077841
min,4017.000000,0.000000,100.000000,0.000000,43.000000,138.000000,16.000000,0.000000,0.000000,20.000000,0.000000,0.000000,100.000000,0.588832
25%,16629.500000,2.000000,123.000000,1.000000,73.000000,157.000000,27.000000,0.000000,0.000000,53.000000,0.000000,1.000000,135.000000,0.894231
50%,17168.000000,5.000000,132.000000,1.000000,85.000000,164.000000,31.000000,0.000000,0.000000,59.000000,0.000000,1.000000,147.000000,0.944954
75%,17664.000000,9.000000,139.000000,1.000000,97.000000,171.000000,35.000000,0.000000,1.000000,66.000000,1.000000,1.000000,161.000000,1.000000
max,18666.000000,24.000000,198.000000,1.000000,180.000000,191.000000,54.000000,1.000000,1.000000,88.000000,1.000000,1.000000,198.000000,1.191667


#### Acotar el dataset

In [175]:
mes = 6
tesis = tesis[tesis["tpo_programa"] <= mes]
mask = tesis.groupby("idPaciente")["tpo_programa"].max() == mes
ids = mask[mask].index
tesis = tesis[tesis["idPaciente"].isin(ids)]

#### Tratamiento de la variable Morisky

In [176]:
# Crear Adherencia_Acum: variable de indice de performance de adherencia al tratamiento hasta el momento t
tesis["Adherencia_Acumulada"] = tesis.groupby("idPaciente")["Adherencia"].expanding().mean().to_list()
# Crear covariable Adherencia_Total no dependiente del tiempo con el performance final de cada paciente
tesis["Adherencia_Total"] = tesis["idPaciente"].map(tesis.groupby("idPaciente")["Adherencia"].mean().to_dict())

#### Crear covariable TAS_media_acum (TAS hasta el tiempo actual)

In [177]:
# Crear TAS_Media_Acum: variable de TAS media hasta el momento t
tesis["TAS_Media_Acumulada"] = tesis.groupby("idPaciente")["TAS"].expanding().mean().to_list()

#### Guardar dataset limpio

In [178]:
# mask = tesis.groupby("idPaciente")["Adherencia_Total", "TAS"].mean()
# mask = mask[mask["Adherencia_Total"] == 1]
# mask.sort_values("TAS", inplace=True, ascending=False)
# mask.reset_index(inplace=True)
# ids_to_remove = mask["idPaciente"].tolist()[:100]
# tesis = tesis[~tesis["idPaciente"].isin(ids_to_remove)]

In [179]:
tesis.groupby("idPaciente")["Adherencia_Total"].mean().value_counts()

1.000000    277
0.857143    104
0.714286     61
0.571429     53
0.428571     28
0.285714     23
0.142857      8
0.000000      6
Name: Adherencia_Total, dtype: int64

In [180]:
tesis.to_csv('Datos/tesis_final.csv', index=False)